In [1]:
import input_data
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [3]:
train, forecast = input_data.get_data(date='20160817')

In [22]:
Tr = train.iloc[::2,:]
Val = train.iloc[1::2,:]
num_feature = Tr.shape[1]-1

In [23]:
trX = Tr.iloc[:,0:num_feature]
trY = Tr.iloc[:,num_feature:num_feature+1]
vaX = Val.iloc[:,0:num_feature]
vaY = Val.iloc[:,num_feature:num_feature+1]
num_data = trX.shape[0]

In [24]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(trY.values)
trYTrans = enc.transform(trY).toarray()
vaYTrans = enc.transform(vaY).toarray()

##  Initialise functions

In [25]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w_h, w_o):
    h = tf.nn.sigmoid(tf.matmul(X, w_h)) # this is a basic mlp, think 2 stacked logistic regressions
    return tf.matmul(h, w_o) # note that we dont take the softmax at the end because our cost fn does that for us

## Initialise Model

In [26]:
num_hidden = 20
X = tf.placeholder("float", [None, num_feature])
Y = tf.placeholder("float", [None, 2])

w_h = init_weights([num_feature, num_hidden]) # create symbolic variables
w_o = init_weights([num_hidden, 2])

In [27]:
py_x = model(X, w_h, w_o)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y)) # compute costs
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer

## Run the model

In [12]:
sess = tf.Session()

In [33]:
train_sample_size = 200
with sess.as_default():
    tf.initialize_all_variables().run()
    for i in range(100):
        for start, end in zip(range(0, num_data, train_sample_size), range(train_sample_size, num_data+1, train_sample_size)):
            sess.run(train_op, feed_dict={X: trX.iloc[start:end,:], Y: trYTrans[start:end]})

In [34]:
with sess.as_default():
    print(sum(sum(vaYTrans * -np.log((py_x.eval(feed_dict={X: vaX.values}) + 1)/2)))/vaYTrans.shape[0])

0.687162443577


In [55]:
with sess.as_default():
    tour_pred = pd.DataFrame((py_x.eval(feed_dict={X: test.values})[:,1] + 1)/2, index=test.index, columns=["probability"])
    input_data.output_pred(tour_pred, 'nn_hidden_%d' % num_hidden, date='20160810')

In [36]:
sess.close()